# CS 7641 Homework 1 - Fall 2020



In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import tree, svm, datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import os

## Loading Datasets

In [3]:
heart_data = pd.read_csv("./data/heart.csv")
heart_data_X = heart_data.drop("target", axis=1)/256
heart_data_y = heart_data.target

heart_train_X, heart_test_X, heart_train_y, heart_test_y = train_test_split(heart_data_X, heart_data_y, test_size=0.3, random_state=100)

In [4]:
# https://scikit-learn.org/stable/modules/tree.html
# https://deepstat.tistory.com/44
# https://jcabelloc.github.io/machine%20learning/2019/02/27/classification-task.html

dtc_heart = tree.DecisionTreeClassifier(random_state=100)
dtc_heart.fit(heart_train_X, heart_train_y)
heart_predict_y = dtc_heart.predict(heart_test_X)
print(f"Accuracy Score: {100* accuracy_score(heart_test_y, heart_predict_y)}%")


Accuracy Score: 76.92307692307693%


## SVM

In [5]:
model = svm.SVC(kernel='rbf', C=30, gamma='auto')
model.fit(heart_train_X, heart_train_y)
model.score(heart_test_X, heart_test_y)

0.7362637362637363

In [6]:
from sklearn.model_selection import cross_val_score

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, make_scorer, classification_report

# https://stackoverflow.com/questions/12632992/gridsearch-for-an-estimator-inside-a-onevsrestclassifier

# model_OVRC = OneVsRestClassifier(svm.LinearSVC(class_weight='balanced', max_iter=10000))
# svm_model = svm.LinearSVC(class_weight='balanced', max_iter=100000)
svm_model = svm.SVC()

parameters = {'C':(0.1, 0.5, 1, 2, 5, 10, 20, 40, 100), 
          "gamma":(0.001, 0.01, 0.1, 0.25, 0.5, 0.75, 1), 
          "kernel":('linear', 'poly', 'rbf')
         }

model = GridSearchCV(svm_model, parameters, cv=5)

model.fit(heart_train_X, heart_train_y)
model_predict = model.predict(heart_test_X)

print(classification_report(heart_test_y, model_predict))


              precision    recall  f1-score   support

           0       0.92      0.74      0.82        46
           1       0.78      0.93      0.85        45

    accuracy                           0.84        91
   macro avg       0.85      0.84      0.83        91
weighted avg       0.85      0.84      0.83        91



## KNN

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

In [7]:
KNN_Model = KNeighborsClassifier(n_neighbors=1).fit(fashion_train_X, fashion_train_y)
KNN_Model_type_1_train_acc = (KNN_Model.predict(fashion_train_X) == fashion_train_y).mean()
print("Training accuracy = ", KNN_Model_type_1_train_acc)

Training accuracy =  1.0
